In [3]:
!pip install mistralai pandas pyarrow dotenv #wget

   ---------------------------------------- 0.0/25.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/25.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/25.8 MB 640.0 kB/s eta 0:00:41
   ---------------------------------------- 0.2/25.8 MB 1.7 MB/s eta 0:00:15
    --------------------------------------- 0.4/25.8 MB 3.1 MB/s eta 0:00:09
    --------------------------------------- 0.6/25.8 MB 3.7 MB/s eta 0:00:07
   - -------------------------------------- 0.9/25.8 MB 4.0 MB/s eta 0:00:07
   - -------------------------------------- 1.0/25.8 MB 4.1 MB/s eta 0:00:07
   - -------------------------------------- 1.2/25.8 MB 3.9 MB/s eta 0:00:07
   -- ------------------------------------- 1.4/25.8 MB 3.9 MB/s eta 0:00:07
   -- ------------------------------------- 1.6/25.8 MB 3.9 MB/s eta 0:00:07
   -- ------------------------------------- 1.8/25.8 MB 4.0 MB/s eta 0:00:06
   --- ------------------------------------ 2.1/25.8 MB 4.2 MB/s eta 0:00:06
   --- ----

In [69]:
import json
import random

# Read the JSONL file
with open('pokemon_battles_dataset.jsonl', 'r') as file:
    data = [json.loads(line) for line in file]
len(data)

295193

In [70]:
import pandas as pd
import json
import random

# Calculate the split
total_samples = len(data)
eval_samples = int(total_samples * 0.005)  # 0.5% for eval
train_samples = total_samples - eval_samples

# Shuffle the data
random.shuffle(data)

# Split the data
train_data = data[:train_samples]
eval_data = data[train_samples:]

# Save train data
with open('pokemon_small_chunk_train.jsonl', 'w') as f:
    for item in train_data:
        json.dump(item, f)
        f.write('\n')

# Save eval data
with open('pokemon_small_chunk_eval.jsonl', 'w') as f:
    for item in eval_data:
        json.dump(item, f)
        f.write('\n')

print(f"Total samples: {total_samples}")
print(f"Train samples: {len(train_data)}")
print(f"Eval samples: {len(eval_data)}")


Total samples: 295193
Train samples: 293718
Eval samples: 1475


In [54]:
# ## script per controllare il formato
# # download the validation and reformat script
# import wget
# url = 'https://raw.githubusercontent.com/mistralai/mistral-finetune/main/utils/validate_data.py'
# wget.download(url)

In [71]:
# validate and reformat the training data
!python reformat_data.py pokemon_small_chunk_eval.jsonl

Skipped 60th sample
Skipped 94th sample
Skipped 95th sample
Skipped 106th sample
Skipped 113th sample
Skipped 185th sample
Skipped 229th sample
Skipped 296th sample
Skipped 300th sample
Skipped 354th sample
Skipped 376th sample
Skipped 412th sample
Skipped 421th sample
Skipped 478th sample
Skipped 523th sample
Skipped 525th sample
Skipped 541th sample
Skipped 562th sample
Skipped 634th sample
Skipped 641th sample
Skipped 642th sample
Skipped 644th sample
Skipped 662th sample
Skipped 667th sample
Skipped 674th sample
Skipped 678th sample
Skipped 707th sample
Skipped 720th sample
Skipped 743th sample
Skipped 744th sample
Skipped 762th sample
Skipped 782th sample
Skipped 798th sample
Skipped 835th sample
Skipped 893th sample
Skipped 993th sample
Skipped 995th sample
Skipped 1032th sample
Skipped 1101th sample
Skipped 1163th sample
Skipped 1179th sample
Skipped 1249th sample
Skipped 1291th sample
Skipped 1330th sample
Skipped 1372th sample
Skipped 1383th sample
Skipped 1387th sample
Skippe

In [72]:
!python reformat_data.py pokemon_small_chunk_train.jsonl

Skipped 11th sample
Skipped 18th sample
Skipped 55th sample
Skipped 70th sample
Skipped 95th sample
Skipped 131th sample
Skipped 147th sample
Skipped 173th sample
Skipped 194th sample
Skipped 205th sample
Skipped 287th sample
Skipped 353th sample
Skipped 443th sample
Skipped 458th sample
Skipped 467th sample
Skipped 501th sample
Skipped 541th sample
Skipped 549th sample
Skipped 558th sample
Skipped 573th sample
Skipped 593th sample
Skipped 621th sample
Skipped 647th sample
Skipped 701th sample
Skipped 797th sample
Skipped 872th sample
Skipped 985th sample
Skipped 1057th sample
Skipped 1078th sample
Skipped 1082th sample
Skipped 1113th sample
Skipped 1150th sample
Skipped 1207th sample
Skipped 1222th sample
Skipped 1310th sample
Skipped 1356th sample
Skipped 1361th sample
Skipped 1392th sample
Skipped 1441th sample
Skipped 1518th sample
Skipped 1529th sample
Skipped 1548th sample
Skipped 1579th sample
Skipped 1681th sample
Skipped 1720th sample
Skipped 1753th sample
Skipped 1792th sampl

In [90]:
import os
from dotenv import load_dotenv
from mistralai.client import MistralClient

# Load environment variables
load_dotenv()
api_key = os.environ.get("MISTRAL_API_KEY")
client = MistralClient(api_key=api_key)

# Upload train chunks
with open("pokemon_chunk_train_1.jsonl", "rb") as f:
    pokemon_chunk_train_1 = client.files.create(file=("pokemon_chunk_train_1.jsonl", f))

print("Train chunk 1 uploaded.")

with open("pokemon_chunk_train_2.jsonl", "rb") as f:
    pokemon_chunk_train_2 = client.files.create(file=("pokemon_chunk_train_2.jsonl", f))

print("Train chunk 2 uploaded.")

with open("pokemon_chunk_train_3.jsonl", "rb") as f:
    pokemon_chunk_train_3 = client.files.create(file=("pokemon_chunk_train_3.jsonl", f))

print("Train chunk 3 uploaded.")

with open("pokemon_chunk_train_4.jsonl", "rb") as f:
    pokemon_chunk_train_4 = client.files.create(file=("pokemon_chunk_train_4.jsonl", f))

print("Train chunk 4 uploaded.")

# Upload eval chunks
with open("pokemon_chunk_eval_1.jsonl", "rb") as f:
    pokemon_chunk_eval_1 = client.files.create(file=("pokemon_chunk_eval_1.jsonl", f))

print("Eval chunk 1 uploaded.")

with open("pokemon_chunk_eval_2.jsonl", "rb") as f:
    pokemon_chunk_eval_2 = client.files.create(file=("pokemon_chunk_eval_2.jsonl", f))

print("Eval chunk 2 uploaded.")

with open("pokemon_chunk_eval_3.jsonl", "rb") as f:
    pokemon_chunk_eval_3 = client.files.create(file=("pokemon_chunk_eval_3.jsonl", f))

print("Eval chunk 3 uploaded.")

with open("pokemon_chunk_eval_4.jsonl", "rb") as f:
    pokemon_chunk_eval_4 = client.files.create(file=("pokemon_chunk_eval_4.jsonl", f))

print("Eval chunk 4 uploaded.")

print("All chunks have been uploaded to Mistral and saved in variables.")



Train chunk 1 uploaded.
Train chunk 2 uploaded.
Train chunk 3 uploaded.
Train chunk 4 uploaded.
Eval chunk 1 uploaded.
Eval chunk 2 uploaded.
Eval chunk 3 uploaded.
Eval chunk 4 uploaded.
All chunks have been uploaded to Mistral and saved in variables.


In [91]:
import json
def pprint(obj):
    print(json.dumps(obj.dict(), indent=4))

In [92]:
pprint(pokemon_chunk_train_1)

{
    "id": "691220a3-7cc1-47bd-848e-d5a38684e770",
    "object": "file",
    "bytes": 364672517,
    "created_at": 1719525683,
    "filename": "pokemon_chunk_train_1.jsonl",
    "purpose": "fine-tune"
}


In [93]:
pprint(pokemon_chunk_eval_1)

{
    "id": "d057842f-63a4-4854-ac7e-a14e5a47d754",
    "object": "file",
    "bytes": 1947509,
    "created_at": 1719526721,
    "filename": "pokemon_chunk_eval_1.jsonl",
    "purpose": "fine-tune"
}


In [104]:
from mistralai.models.jobs import TrainingParameters

dry_run_job = client.jobs.create(
    model="open-mistral-7b",
    training_files=[pokemon_chunk_train_1.id, pokemon_chunk_train_2.id, pokemon_chunk_train_3.id, pokemon_chunk_train_4.id],
    hyperparameters=TrainingParameters(
        training_steps=3000,
        learning_rate=0.0001,
    ),
    dry_run=True,
)
print(dry_run_job)

object='job.metadata' training_steps=3000 train_tokens_per_step=131072 data_tokens=363881282 train_tokens=393216000 epochs=1.0806 expected_duration_seconds=21000


In [105]:
from mistralai.models.jobs import TrainingParameters
from mistralai.models.jobs import WandbIntegrationIn

WANDB_API_KEY = os.environ.get("WANDB_API_KEY")

created_jobs = client.jobs.create(
    model= "mistral-small-latest", #"open-mistral-7b",
    training_files=[pokemon_chunk_train_1.id, pokemon_chunk_train_2.id, pokemon_chunk_train_3.id, pokemon_chunk_train_4.id],
    validation_files=[pokemon_chunk_eval_1.id, pokemon_chunk_eval_2.id, pokemon_chunk_eval_3.id, pokemon_chunk_eval_4.id],
    hyperparameters=TrainingParameters(
        training_steps=3000,
        learning_rate=0.0001, 
        ),
    integrations=[
        WandbIntegrationIn(
            project="test_mistral_api",
            run_name="pokemon_test_run",
            api_key=WANDB_API_KEY,
        ).dict()
    ],
)

MistralAPIException: Status: 402. Message: {"detail": "Launching a fine tuning job requires credits in your wallet. You can add credits via the console. Current monthly usage: 54.8502 EUR. Job cost: 1494.2208 EUR. Wallet balance: 100.0 EUR."}

In [97]:
pprint(created_jobs)

{
    "id": "e220572d-23aa-4115-8c75-9d08b08dafb6",
    "hyperparameters": {
        "training_steps": 100,
        "learning_rate": 0.0001
    },
    "fine_tuned_model": null,
    "model": "mistral-small-latest",
    "status": "QUEUED",
    "job_type": "FT",
    "created_at": 1719531517,
    "modified_at": 1719531517,
    "training_files": [
        "691220a3-7cc1-47bd-848e-d5a38684e770",
        "c3445bee-3f37-44ac-80ee-f2896be55d0e",
        "de7c5776-034e-4980-95af-20dca2a2f411",
        "e03f89a1-ea90-4e1c-b767-1fa87c8d7c5b"
    ],
    "validation_files": [
        "1b8bef0d-5b64-4e35-816a-258fd33ec7be",
        "26a1e757-d298-4e2c-b523-4714dc8220ed",
        "d057842f-63a4-4854-ac7e-a14e5a47d754",
        "e1502f5b-a711-47bc-b0da-a56e1962e5bd"
    ],
    "object": "job",
    "integrations": [
        {
            "type": "wandb",
            "project": "test_mistral_api",
            "name": null,
            "run_name": "pokemon_test_run"
        }
    ]
}


In [98]:
import time

retrieved_job = client.jobs.retrieve(created_jobs.id)
while retrieved_job.status in ["RUNNING", "QUEUED"]:
    retrieved_job = client.jobs.retrieve(created_jobs.id)
    pprint(retrieved_job)
    print(f"Job is {retrieved_job.status}, waiting 10 seconds")
    time.sleep(10)

{
    "id": "e220572d-23aa-4115-8c75-9d08b08dafb6",
    "hyperparameters": {
        "training_steps": 100,
        "learning_rate": 0.0001
    },
    "fine_tuned_model": null,
    "model": "mistral-small-latest",
    "status": "RUNNING",
    "job_type": "FT",
    "created_at": 1719531517,
    "modified_at": 1719531518,
    "training_files": [
        "691220a3-7cc1-47bd-848e-d5a38684e770",
        "c3445bee-3f37-44ac-80ee-f2896be55d0e",
        "de7c5776-034e-4980-95af-20dca2a2f411",
        "e03f89a1-ea90-4e1c-b767-1fa87c8d7c5b"
    ],
    "validation_files": [
        "1b8bef0d-5b64-4e35-816a-258fd33ec7be",
        "26a1e757-d298-4e2c-b523-4714dc8220ed",
        "d057842f-63a4-4854-ac7e-a14e5a47d754",
        "e1502f5b-a711-47bc-b0da-a56e1962e5bd"
    ],
    "object": "job",
    "integrations": [
        {
            "type": "wandb",
            "project": "test_mistral_api",
            "name": null,
            "run_name": "pokemon_test_run"
        }
    ],
    "events": [
     

In [99]:
# List jobs
jobs = client.jobs.list()
pprint(jobs)

{
    "data": [
        {
            "id": "e220572d-23aa-4115-8c75-9d08b08dafb6",
            "hyperparameters": {
                "training_steps": 100,
                "learning_rate": 0.0001
            },
            "fine_tuned_model": "ft:mistral-small-latest:f03cfd29:20240627:e220572d",
            "model": "mistral-small-latest",
            "status": "SUCCESS",
            "job_type": "FT",
            "created_at": 1719531517,
            "modified_at": 1719534390,
            "training_files": [
                "691220a3-7cc1-47bd-848e-d5a38684e770",
                "c3445bee-3f37-44ac-80ee-f2896be55d0e",
                "de7c5776-034e-4980-95af-20dca2a2f411",
                "e03f89a1-ea90-4e1c-b767-1fa87c8d7c5b"
            ],
            "validation_files": [
                "1b8bef0d-5b64-4e35-816a-258fd33ec7be",
                "e1502f5b-a711-47bc-b0da-a56e1962e5bd",
                "26a1e757-d298-4e2c-b523-4714dc8220ed",
                "d057842f-63a4-4854-ac7e-a14e5

In [100]:
# Retrieve a jobs
retrieved_jobs = client.jobs.retrieve(created_jobs.id)
pprint(retrieved_jobs)

{
    "id": "e220572d-23aa-4115-8c75-9d08b08dafb6",
    "hyperparameters": {
        "training_steps": 100,
        "learning_rate": 0.0001
    },
    "fine_tuned_model": "ft:mistral-small-latest:f03cfd29:20240627:e220572d",
    "model": "mistral-small-latest",
    "status": "SUCCESS",
    "job_type": "FT",
    "created_at": 1719531517,
    "modified_at": 1719534390,
    "training_files": [
        "691220a3-7cc1-47bd-848e-d5a38684e770",
        "c3445bee-3f37-44ac-80ee-f2896be55d0e",
        "de7c5776-034e-4980-95af-20dca2a2f411",
        "e03f89a1-ea90-4e1c-b767-1fa87c8d7c5b"
    ],
    "validation_files": [
        "1b8bef0d-5b64-4e35-816a-258fd33ec7be",
        "e1502f5b-a711-47bc-b0da-a56e1962e5bd",
        "26a1e757-d298-4e2c-b523-4714dc8220ed",
        "d057842f-63a4-4854-ac7e-a14e5a47d754"
    ],
    "object": "job",
    "integrations": [
        {
            "type": "wandb",
            "project": "test_mistral_api",
            "name": null,
            "run_name": "pokemon_

In [101]:
prompt = "I am a Pok\u00e9mon trainer starting the turn 34.\nYou've this pok\u00e9mon in the field: Porygon2 {'names': 'Porygon2', 'liv': 'L82', 'moves': ['Toxic', 'Recover'], 'abilit\u00e0': ''}.\nYour opponent have this pok\u00e9mon: Nidoqueen.\nYou have these switches available:\n- Watchog | {'names': 'Watchog', 'liv': 'L94', 'moves': ['Hypnosis'], 'abilit\u00e0': ''} | fainted\n- Heracross | {'names': 'Heracross', 'liv': 'L80', 'moves': ['Earthquake', 'Stone Edge'], 'abilit\u00e0': ''}\n- Tropius | {'names': 'Tropius', 'liv': 'L94', 'moves': ['Leech Seed', 'Substitute', 'Protect', 'Air Slash'], 'abilit\u00e0': ''} | fainted\n- Lunatone | {'names': 'Lunatone', 'liv': 'L92', 'moves': ['Calm Mind', 'Earth Power'], 'abilit\u00e0': ''} | fainted\n- Azumarill | {'names': 'Azumarill', 'liv': 'L85', 'moves': ['Belly Drum', 'Aqua Jet', 'Waterfall'], 'abilit\u00e0': ''} | fainted."
question = "|start\n|switch|p1a: Watchog|Watchog, L94, M|265/265\n|switch|p2a: Azumarill|Azumarill, L85, F|309/309\n|turn|1\n|\n|move|p1a: Watchog|Hypnosis|p2a: Azumarill|[miss]\n|-miss|p1a: Watchog|p2a: Azumarill\n|move|p2a: Azumarill|Superpower|p1a: Watchog\n|-supereffective|p1a: Watchog\n|-damage|p1a: Watchog|0 fnt\n|-unboost|p2a: Azumarill|atk|1\n|-unboost|p2a: Azumarill|def|1\n|faint|p1a: Watchog\n|\n|upkeep\n|\n|switch|p1a: Heracross|Heracross, L80, F|259/259\n|turn|2\n|\n|switch|p1a: Tropius|Tropius, L94, M|336/336\n|move|p2a: Azumarill|Superpower|p1a: Tropius\n|-resisted|p1a: Tropius\n|-damage|p1a: Tropius|288/336\n|-unboost|p2a: Azumarill|atk|1\n|-unboost|p2a: Azumarill|def|1\n|\n|upkeep\n|turn|3\n|\n|switch|p2a: Vanilluxe|Vanilluxe, L90, F|274/274\n|move|p1a: Tropius|Leech Seed|p2a: Vanilluxe|[miss]\n|-miss|p1a: Tropius|p2a: Vanilluxe\n|\n|upkeep\n|turn|4\n|\n|switch|p1a: Lunatone|Lunatone, L92|278/278\n|move|p2a: Vanilluxe|Autotomize|p2a: Vanilluxe\n|-boost|p2a: Vanilluxe|spe|2\n|-start|p2a: Vanilluxe|Autotomize\n|\n|upkeep\n|turn|5\n|\n|move|p2a: Vanilluxe|Flash Cannon|p1a: Lunatone\n|-supereffective|p1a: Lunatone\n|-damage|p1a: Lunatone|106/278\n|-damage|p2a: Vanilluxe|247/274|[from] item: Life Orb\n|move|p1a: Lunatone|Calm Mind|p1a: Lunatone\n|-boost|p1a: Lunatone|spa|1\n|-boost|p1a: Lunatone|spd|1\n|\n|-heal|p1a: Lunatone|123/278|[from] item: Leftovers\n|upkeep\n|turn|6\n|\n|switch|p1a: Porygon2|Porygon2, L82|274/274\n|-ability|p1a: Porygon2|Download|boost\n|-boost|p1a: Porygon2|atk|1\n|move|p2a: Vanilluxe|Flash Cannon|p1a: Porygon2\n|-damage|p1a: Porygon2|212/274\n|-damage|p2a: Vanilluxe|220/274|[from] item: Life Orb\n|\n|upkeep\n|turn|7\n|\n|move|p2a: Vanilluxe|Ice Beam|p1a: Porygon2\n|-damage|p1a: Porygon2|95/274\n|-damage|p2a: Vanilluxe|193/274|[from] item: Life Orb\n|move|p1a: Porygon2|Toxic|p2a: Vanilluxe\n|-status|p2a: Vanilluxe|tox\n|\n|-damage|p2a: Vanilluxe|176/274 tox|[from] psn\n|upkeep\n|turn|8\n|\n|switch|p1a: Azumarill|Azumarill, L85, F|308/308\n|move|p2a: Vanilluxe|Ice Beam|p1a: Azumarill\n|-resisted|p1a: Azumarill\n|-damage|p1a: Azumarill|213/308\n|-damage|p2a: Vanilluxe|149/274 tox|[from] item: Life Orb\n|\n|-damage|p2a: Vanilluxe|115/274 tox|[from] psn\n|upkeep\n|turn|9\n|\n|move|p2a: Vanilluxe|Ice Beam|p1a: Azumarill\n|-resisted|p1a: Azumarill\n|-damage|p1a: Azumarill|119/308\n|-enditem|p1a: Azumarill|Sitrus Berry|[eat]\n|-heal|p1a: Azumarill|196/308|[from] item: Sitrus Berry\n|-damage|p2a: Vanilluxe|88/274 tox|[from] item: Life Orb\n|move|p1a: Azumarill|Belly Drum|p1a: Azumarill\n|-damage|p1a: Azumarill|42/308\n|-setboost|p1a: Azumarill|atk|6|[from] move: Belly Drum\n|\n|-damage|p2a: Vanilluxe|37/274 tox|[from] psn\n|upkeep\n|turn|10\n|\n|move|p1a: Azumarill|Aqua Jet|p2a: Vanilluxe\n|-crit|p2a: Vanilluxe\n|-damage|p2a: Vanilluxe|0 fnt\n|faint|p2a: Vanilluxe\n|\n|upkeep\n|\n|switch|p2a: Beheeyem|Beheeyem, L89, M|278/278\n|turn|11\n|\n|move|p1a: Azumarill|Waterfall|p2a: Beheeyem\n|-damage|p2a: Beheeyem|0 fnt\n|faint|p2a: Beheeyem\n|\n|upkeep\n|\n|switch|p2a: Azumarill|Azumarill, L85, F|309/309\n|turn|12\n\n|\n|move|p2a: Azumarill|Aqua Jet|p1a: Azumarill\n|-resisted|p1a: Azumarill\n|-damage|p1a: Azumarill|0 fnt\n|faint|p1a: Azumarill\n|\n|upkeep\n|\n|switch|p1a: Porygon2|Porygon2, L82|95/274\n|-ability|p1a: Porygon2|Download|boost\n|-boost|p1a: Porygon2|spa|1\n|turn|13\n|\n|move|p2a: Azumarill|Aqua Jet|p1a: Porygon2\n|-damage|p1a: Porygon2|40/274\n|move|p1a: Porygon2|Recover|p1a: Porygon2\n|-heal|p1a: Porygon2|177/274\n|\n|upkeep\n|turn|14\n|\n|move|p2a: Azumarill|Aqua Jet|p1a: Porygon2\n|-damage|p1a: Porygon2|125/274\n|move|p1a: Porygon2|Toxic|p2a: Azumarill\n|-status|p2a: Azumarill|tox\n|\n|-damage|p2a: Azumarill|290/309 tox|[from] psn\n|upkeep\n|turn|15\n|\n|move|p2a: Azumarill|Aqua Jet|p1a: Porygon2\n|-damage|p1a: Porygon2|71/274\n|move|p1a: Porygon2|Recover|p1a: Porygon2\n|-heal|p1a: Porygon2|208/274\n|\n|-damage|p2a: Azumarill|252/309 tox|[from] psn\n|upkeep\n|turn|16\n|\n|switch|p2a: Plusle|Plusle, L90, F|254/254\n|move|p1a: Porygon2|Recover|p1a: Porygon2\n|-heal|p1a: Porygon2|274/274\n|\n|upkeep\n|turn|17\n|\n|switch|p1a: Lunatone|Lunatone, L92|123/278\n|move|p2a: Plusle|Nasty Plot|p2a: Plusle\n|-boost|p2a: Plusle|spa|2\n|\n|-heal|p1a: Lunatone|140/278|[from] item: Leftovers\n|upkeep\n|turn|18\n|\n|move|p2a: Plusle|Hidden Power|p1a: Lunatone\n|-damage|p1a: Lunatone|46/278\n|move|p1a: Lunatone|Earth Power|p2a: Plusle\n|-supereffective|p2a: Plusle\n|-damage|p2a: Plusle|84/254\n|\n|-heal|p2a: Plusle|99/254|[from] item: Leftovers\n|-heal|p1a: Lunatone|63/278|[from] item: Leftovers\n|upkeep\n|turn|19\n|\n|move|p2a: Plusle|Hidden Power|p1a: Lunatone\n|-damage|p1a: Lunatone|0 fnt\n|faint|p1a: Lunatone\n|\n|-heal|p2a: Plusle|114/254|[from] item: Leftovers\n|upkeep\n|\n|switch|p1a: Heracross|Heracross, L80, F|259/259\n|turn|20\n|inactiveoff|Battle timer is now OFF.\n|\n|move|p1a: Heracross|Earthquake|p2a: Plusle\n|-supereffective|p2a: Plusle\n|-damage|p2a: Plusle|0 fnt\n|faint|p2a: Plusle\n|-ability|p1a: Heracross|Moxie|boost\n|-boost|p1a: Heracross|atk|1\n|\n|upkeep\n|\n|switch|p2a: Azumarill|Azumarill, L85, F|252/309 tox\n|turn|21\n|\n|switch|p1a: Porygon2|Porygon2, L82|274/274\n|-ability|p1a: Porygon2|Download|boost\n|-boost|p1a: Porygon2|spa|1\n|move|p2a: Azumarill|Superpower|p1a: Porygon2\n|-supereffective|p1a: Porygon2\n|-damage|p1a: Porygon2|44/274\n|-unboost|p2a: Azumarill|atk|1\n|-unboost|p2a: Azumarill|def|1\n|\n|-damage|p2a: Azumarill|233/309 tox|[from] psn\n|upkeep\n|turn|22\n|\n|switch|p1a: Tropius|Tropius, L94, M|288/336\n|move|p2a: Azumarill|Superpower|p1a: Tropius\n|-resisted|p1a: Tropius\n|-damage|p1a: Tropius|240/336\n|-unboost|p2a: Azumarill|atk|1\n|-unboost|p2a: Azumarill|def|1\n|\n|-damage|p2a: Azumarill|195/309 tox|[from] psn\n|upkeep\n|turn|23\n|\n|move|p1a: Tropius|Substitute|p1a: Tropius\n|-start|p1a: Tropius|Substitute\n|-damage|p1a: Tropius|156/336\n|-enditem|p1a: Tropius|Sitrus Berry|[eat]\n|-heal|p1a: Tropius|240/336|[from] item: Sitrus Berry\n|move|p2a: Azumarill|Superpower|p1a: Tropius\n|-resisted|p1a: Tropius\n|-activate|p1a: Tropius|Substitute|[damage]\n|-unboost|p2a: Azumarill|atk|1\n|-unboost|p2a: Azumarill|def|1\n|\n|-damage|p2a: Azumarill|138/309 tox|[from] psn\n|-item|p1a: Tropius|Sitrus Berry|[from] ability: Harvest\n|upkeep\n|turn|24\n|\n|move|p1a: Tropius|Leech Seed|p2a: Azumarill\n|-start|p2a: Azumarill|move: Leech Seed\n|move|p2a: Azumarill|Superpower|p1a: Tropius\n|-resisted|p1a: Tropius\n|-activate|p1a: Tropius|Substitute|[damage]\n|-unboost|p2a: Azumarill|atk|1\n|-unboost|p2a: Azumarill|def|1\n|\n|-damage|p2a: Azumarill|100/309 tox|[from] Leech Seed|[of] p1a: Tropius\n|-heal|p1a: Tropius|278/336|[silent]\n|-damage|p2a: Azumarill|24/309 tox|[from] psn\n|upkeep\n|turn|25\n|\n|move|p1a: Tropius|Protect|p1a: Tropius\n|-singleturn|p1a: Tropius|Protect\n|move|p2a: Azumarill|Superpower|p1a: Tropius\n|-activate|p1a: Tropius|move: Protect\n|\n|-damage|p2a: Azumarill|0 fnt|[from] Leech Seed|[of] p1a: Tropius\n|-heal|p1a: Tropius|302/336|[silent]\n|faint|p2a: Azumarill\n|upkeep\n|\n|switch|p2a: Mightyena|Mightyena, L94, M|284/284\n|-ability|p2a: Mightyena|Intimidate|boost\n|-immune|p1a: Tropius\n|turn|26\n|\n|move|p2a: Mightyena|Fire Fang|p1a: Tropius\n|-supereffective|p1a: Tropius\n|-end|p1a: Tropius|Substitute\n|move|p1a: Tropius|Leech Seed|p2a: Mightyena\n|-start|p2a: Mightyena|move: Leech Seed\n|\n|-damage|p2a: Mightyena|249/284|[from] Leech Seed|[of] p1a: Tropius\n|-heal|p1a: Tropius|336/336|[silent]\n|upkeep\n|turn|27\n|\n|move|p1a: Tropius|Protect|p1a: Tropius\n|-singleturn|p1a: Tropius|Protect\n|move|p2a: Mightyena|Fire Fang|p1a: Tropius\n|-activate|p1a: Tropius|move: Protect\n|\n|-damage|p2a: Mightyena|214/284|[from] Leech Seed|[of] p1a: Tropius\n|upkeep\n|turn|28\n|\n|move|p2a: Mightyena|Fire Fang|p1a: Tropius\n|-supereffective|p1a: Tropius\n|-crit|p1a: Tropius\n|-damage|p1a: Tropius|52/336\n|-enditem|p1a: Tropius|Sitrus Berry|[eat]\n|-heal|p1a: Tropius|136/336|[from] item: Sitrus Berry\n|move|p1a: Tropius|Substitute|p1a: Tropius\n|-start|p1a: Tropius|Substitute\n|-damage|p1a: Tropius|52/336\n|\n|-damage|p2a: Mightyena|179/284|[from] Leech Seed|[of] p1a: Tropius\n|-heal|p1a: Tropius|87/336|[silent]\n|upkeep\n|turn|29\n|\n|move|p1a: Tropius|Protect|p1a: Tropius\n|-singleturn|p1a: Tropius|Protect\n|move|p2a: Mightyena|Fire Fang|p1a: Tropius\n|-activate|p1a: Tropius|move: Protect\n|\n|-damage|p2a: Mightyena|144/284|[from] Leech Seed|[of] p1a: Tropius\n|-heal|p1a: Tropius|122/336|[silent]\n|-item|p1a: Tropius|Sitrus Berry|[from] ability: Harvest\n|upkeep\n|-enditem|p1a: Tropius|Sitrus Berry|[eat]\n|-heal|p1a: Tropius|206/336|[from] item: Sitrus Berry\n|turn|30\n|\n|move|p2a: Mightyena|Fire Fang|p1a: Tropius\n|-supereffective|p1a: Tropius\n|-end|p1a: Tropius|Substitute\n|move|p1a: Tropius|Substitute|p1a: Tropius\n|-start|p1a: Tropius|Substitute\n|-damage|p1a: Tropius|122/336\n|\n|-damage|p2a: Mightyena|109/284|[from] Leech Seed|[of] p1a: Tropius\n|-heal|p1a: Tropius|157/336|[silent]\n|upkeep\n|turn|31\n|\n|move|p1a: Tropius|Protect|p1a: Tropius\n|-singleturn|p1a: Tropius|Protect\n|move|p2a: Mightyena|Fire Fang|p1a: Tropius\n|-activate|p1a: Tropius|move: Protect\n|\n|-damage|p2a: Mightyena|74/284|[from] Leech Seed|[of] p1a: Tropius\n|-heal|p1a: Tropius|192/336|[silent]\n|-item|p1a: Tropius|Sitrus Berry|[from] ability: Harvest\n|upkeep\n|turn|32\n|\n|move|p2a: Mightyena|Fire Fang|p1a: Tropius\n|-supereffective|p1a: Tropius\n|-end|p1a: Tropius|Substitute\n|move|p1a: Tropius|Air Slash|p2a: Mightyena\n|-damage|p2a: Mightyena|0 fnt\n|faint|p2a: Mightyena\n|\n|upkeep\n|\n|switch|p2a: Nidoqueen|Nidoqueen, L82, F|282/282\n|turn|33\n|\n|move|p2a: Nidoqueen|Sludge Wave|p1a: Tropius\n|-supereffective|p1a: Tropius\n|-damage|p1a: Tropius|0 fnt\n|faint|p1a: Tropius\n|\n|upkeep\n|\n|switch|p1a: Porygon2|Porygon2, L82|44/274\n|-ability|p1a: Porygon2|Download|boost\n|-boost|p1a: Porygon2|spa|1"

In [102]:
## inferenza
from mistralai.models.chat_completion import ChatMessage

chat_response = client.chat(
    model=retrieved_jobs.fine_tuned_model,
    messages=[
        ChatMessage(role="system", content=prompt),
        ChatMessage(role="user", content=question)
    ]
)
pprint(chat_response)

{
    "id": "cd550e7159994ee9a570ad1fbc66fbb6",
    "object": "chat.completion",
    "created": 1719552256,
    "model": "ft:mistral-small-latest:f03cfd29:20240627:e220572d",
    "choices": [
        {
            "index": 0,
            "message": {
                "role": "assistant",
                "content": "move|Recover",
                "name": null,
                "tool_calls": null,
                "tool_call_id": null
            },
            "finish_reason": "stop"
        }
    ],
    "usage": {
        "prompt_tokens": 5913,
        "total_tokens": 5917,
        "completion_tokens": 4
    }
}
